# [577. Employee Bonus](https://leetcode.com/problems/employee-bonus/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employee

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| empId       | int     |
| name        | varchar |
| supervisor  | int     |
| salary      | int     |
+-------------+---------+</pre>
empId is the column with unique values for this table.
Each row of this table indicates the name and the ID of an employee in addition to their salary and the id of their manager.
 

Table: Bonus

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| empId       | int  |
| bonus       | int  |
+-------------+------+</pre>
empId is the column of unique values for this table.
empId is a foreign key (reference column) to empId from the Employee table.
Each row of this table contains the id of an employee and their respective bonus.
 

Write a solution to report the name and bonus amount of each employee with a bonus less than 1000.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
<pre>+-------+--------+------------+--------+
| empId | name   | supervisor | salary |
+-------+--------+------------+--------+
| 3     | Brad   | null       | 4000   |
| 1     | John   | 3          | 1000   |
| 2     | Dan    | 3          | 2000   |
| 4     | Thomas | 3          | 4000   |
+-------+--------+------------+--------+</pre>
Bonus table:
<pre>+-------+-------+
| empId | bonus |
+-------+-------+
| 2     | 500   |
| 4     | 2000  |
+-------+-------+</pre>
Output: 
<pre>+------+-------+
| name | bonus |
+------+-------+
| Brad | null  |
| John | null  |
| Dan  | 500   |
+------+-------+</pre>

In [1]:
# Pandas Schema

import pandas as pd

data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype(
    {'empId': 'Int64', 'name': 'object', 'supervisor': 'Int64', 'salary': 'Int64'})
data2 = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data2, columns=['empId', 'bonus']).astype({'empId': 'Int64', 'bonus': 'Int64'})

In [2]:
# to pyspark schema

from pyspark.sql.types import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

schema_employee = StructType([
    StructField("empId", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("supervisor", IntegerType(), True),
    StructField("salary", IntegerType(), True)
])

employee_df = spark.createDataFrame(data, schema_employee)
bonus_df = spark.createDataFrame(bonus)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 12:04:02 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 12:04:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 12:04:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
employee_df.show(truncate=False)

+-----+------+----------+------+
|empId|name  |supervisor|salary|
+-----+------+----------+------+
|3    |Brad  |null      |4000  |
|1    |John  |3         |1000  |
|2    |Dan   |3         |2000  |
|4    |Thomas|3         |4000  |
+-----+------+----------+------+


In [4]:
employee_df.printSchema()

root
 |-- empId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- supervisor: integer (nullable = true)
 |-- salary: integer (nullable = true)


In [5]:
bonus_df.show(truncate=False)

+-----+-----+
|empId|bonus|
+-----+-----+
|2    |500  |
|4    |2000 |
+-----+-----+


In [6]:
# In spark Dataframe

import pyspark.sql.functions as F

employee_df.join(bonus_df, on=['empId'], how='left') \
    .filter((F.col('bonus') < 1000) | (F.col('bonus').isNull())) \
    .select('name', 'bonus').show()

+----+-----+
|name|bonus|
+----+-----+
|Brad| null|
|John| null|
| Dan|  500|
+----+-----+


In [7]:
# In spark SQL

employee_df.createOrReplaceTempView('employee')
bonus_df.createOrReplaceTempView('bonus')

spark.sql('''
select name,bonus from employee
left join bonus
on employee.empId=bonus.empId
where bonus is null or bonus<1000;
''').show()

+----+-----+
|name|bonus|
+----+-----+
|Brad| null|
|John| null|
| Dan|  500|
+----+-----+


In [8]:
spark.stop()